# Auxilary Classifier GAN (ACGAN)

Referecnce : https://github.com/eriklindernoren/PyTorch-GAN

In [ ]:
import argparse
import os
import numpy as np
import math
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Use MNIST Dataset & DataLoader

In [ ]:
bs =128

dataset = datasets.MNIST("./mnist", train=True, download=True, 
                         transform=transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=True, drop_last=True)

In [ ]:
batch = next(iter(dataloader))
len(batch)
print(batch[0].shape, batch[1])

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(20,10))
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()
    
imshow(torchvision.utils.make_grid(batch[0]))

## Define Generator & Discriminator
### Generator architecture
input random vector: 100 dim
* Embedding: embeding labels to 100 dim
* Linear: out_features 128 * 8 * 8
* BatchNorm2d
* Upsample: scale_factor 2
* Conv2d: out_channel: 128, kernel size 3, stride 1, padding 1
* BatchNorm2d
* LeakyReLU: 0.2
* Upsample: scale_factor 2
* Conv2d: out_channel: 64, kernel size 3, stride 1, padding 1
* BatchNorm
* LeakyReLU: 0.2
* Conv2d: out_channel: 1, kernel size 3, stride 1, padding 1
* Tanh

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        #### Implement Here ####
        
        
    def forward(self, z, labels):
        #### Implement Here ####
        ## noise, labels -> noise: 100, label: 1 dim -> 100 dim 
        ## noise * label

        
        return img

### Discriminator architecture
input: [1 , 32 , 32] image  
* Conv2d: out_channel: 16, kernel size 3, stride 2, padding 1
* LeakyReLU: 0.2
* Dropout: 0.25
* Conv2d: out_channel: 32, kernel size 3, stride 2, padding 1
* LeakyReLU: 0.2
* Dropout: 0.25
* BatchNorm2d
* Conv2d: out_channel: 64, kernel size 3, stride 2, padding 1
* LeakyReLU: 0.2
* Dropout: 0.25
* BatchNorm2d
* Conv2d: out_channel: 128, kernel size 3, stride 2, padding 1
* LeakyReLU: 0.2
* Dropout: 0.25
* BatchNorm2d
* two linear layers: one for adversarial loss, one for classification
- Linear(Val) : output 1, activation : sigmoid
- Linear(classification) : output 10, no activation

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        #### Implement Here ####

        
        
    def make_block(self, in_channel, out_channel, bn= True):
        #### Implement Here ####
        layers = [

        ]
        if bn:
            layers.append()
        
        return nn.Sequential(*layers) 
        
        
    def forward(self, img):
        ## fill here

        
        return validity, labels

In [ ]:
G = Generator()

input_sample = torch.randn(10,100)
labels = torch.Tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).long()
output = G(input_sample, labels)
output.shape

In [ ]:
D = Discriminator()
validity, pred_label = D(output)
print(validity.shape)
print(pred_label.shape)

## Weight Initialization

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1: # Conv2d layer weights init
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1: # BatchNorm2d layer weights init
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


# weight initialize
G.apply(weights_init_normal)
D.apply(weights_init_normal)

## Define loss & Optimizer & model

In [ ]:
# Loss function
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

## optimizer fill here
optimizer_G = torch.optim.Adam(G.parameters(), lr=0.00002, betas=(0.5, 0.9999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=0.00002, betas=(0.5, 0.9999))

## Start Training

In [ ]:
G, D = G.to(device), D.to(device)
os.makedirs("./acgan_images", exist_ok=True)
epochs = 200

In [ ]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = torch.Tensor(np.random.normal(0, 1, (n_row ** 2, 100))).to(device)
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = torch.LongTensor(labels).to(device)
    gen_imgs = G(z, labels)
    save_image(gen_imgs, "acgan_images/%06d.png" % batches_done, nrow=n_row, normalize=True)
    imshow(torchvision.utils.make_grid(gen_imgs.cpu(), nrow = n_row))

In [ ]:
for epoch in range(epochs):
    for i, (real_imgs, labels) in enumerate(dataloader):

        batch_size = real_imgs.shape[0]

        # Configure input
        real_imgs = real_imgs.to(device)
        real_labels = labels.to(device)
        
        ones = torch.ones((real_imgs.size(0), 1)).to(device)
        zeros = torch.zeros((real_imgs.size(0), 1)).to(device)
        
        ## z random sample
        ## label random sample 0-9 int
        z = torch.Tensor(np.random.normal(0, 1, (batch_size, 100))).to(device)
        gen_labels = torch.LongTensor(np.random.randint(0, 10, batch_size)).to(device)
        
        # -----------------
        #  Train Generator
        # -----------------
        #### Implement Here ####
        
        ## generator, discriminator output

        
        ## g_loss = adversarial_loss + classification loss


        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        #### Implement Here ####
        
        # Loss for real images : adversarial_loss + classification loss

        
        # Loss for fake images : adversarial_loss + classification loss

        
        # Total discriminator loss

        
        # Calculate discriminator accuracy
        pred = np.concatenate([real_labels_pred.detach().cpu().numpy(), fake_labels_pred.detach().cpu().numpy()], axis=0)
        gt = np.concatenate([labels.detach().cpu().numpy(), gen_labels.detach().cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        batches_done = epoch * len(dataloader) + i
        if batches_done % 100 == 0:     
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
                % (epoch, 200, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
            )
            
        if batches_done % 2000 == 0:
            sample_image(n_row=10, batches_done=batches_done)